# Modeling Fixed rate bonds

Let's consider a bond with a par value of 100 that pays 6% coupon semi-annually issued on Jan, 15, 2015 and set to mature on Jan, 15, 2016. The bond will pay a coupon on July 15, 2015 and Jan 15, 2016. The par amount of 100 will also be repaid on Jan 15, 2016.

Let's assume that we know the spot rates of the treasury as of Jan 15, 2015. The annualized spot rates are 0.5% for 6 months and 0.7% for 1 year. Let's calculate the fair value of this bond.

In [1]:
# without QL
3/pow(1+0.005,0.5) + (100 + 3)/(1+0.007)

105.27653992490681

In [4]:
# with QL
import QuantLib as ql
today = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = today
spotDates = [ql.Date(15,1,2015), ql.Date(15,7,2015), ql.Date(15, 1,2016)]
spotRates = [0.0, 0.005, 0.007]
dayCount = ql.Thirty360()
calendar = ql.UnitedStates()
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation, compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)

So far we have created the term structure and the variables are rather self explanatory. Now let's construct the fixed rate bond.

In [6]:
issueDate = ql.Date(15,1,2015)
maturityDate = ql.Date(15, 1, 2016)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates()
businessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule(issueDate, maturityDate, tenor,
                      calendar, businessConvention, businessConvention, dateGeneration,
                      monthEnd)
list(schedule)

[Date(15,1,2015), Date(15,7,2015), Date(15,1,2016)]

In [7]:
# now let's build the coupon
dayCount = ql.Thirty360()
couponRate = .06
coupons = [couponRate]

# now let's construct the FixedRateBond
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)

In [8]:
# create a bond engine with the term structure as input
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

In [9]:
# finally the price
fixedRateBond.NPV()

105.27653992490683